In [9]:
import pandas as pd

# CSV の読み込み
df = pd.read_csv("games.csv", encoding="shift_jis")


# 勝敗判定
df["winner"] = None
df["loser"] = None
df["is_draw"] = df["home_team_score"] == df["away_team_score"]

# ホーム勝ち
home_win = df["home_team_score"] > df["away_team_score"]
df.loc[home_win, "winner"] = df.loc[home_win, "home_team_id"]
df.loc[home_win, "loser"]  = df.loc[home_win, "away_team_id"]

# アウェイ勝ち
away_win = df["home_team_score"] < df["away_team_score"]
df.loc[away_win, "winner"] = df.loc[away_win, "away_team_id"]
df.loc[away_win, "loser"]  = df.loc[away_win, "home_team_id"]

# -------------------------------
# チーム別の集計
# -------------------------------

# 勝ち数
wins = df.groupby(["year_id", "winner"]).size().reset_index(name="wins")
wins = wins.rename(columns={"winner": "team_id"})

# 負け数
losses = df.groupby(["year_id", "loser"]).size().reset_index(name="losses")
losses = losses.rename(columns={"loser": "team_id"})

# 引き分け数
draws_home = df[df["is_draw"]].groupby(["year_id", "home_team_id"]).size().reset_index(name="draws")
draws_home = draws_home.rename(columns={"home_team_id": "team_id"})

draws_away = df[df["is_draw"]].groupby(["year_id", "away_team_id"]).size().reset_index(name="draws")
draws_away = draws_away.rename(columns={"away_team_id": "team_id"})

draws = pd.concat([draws_home, draws_away], ignore_index=True)
draws = draws.groupby(["year_id", "team_id"])["draws"].sum().reset_index()

# -------------------------------
# 勝敗表をまとめる
# -------------------------------
standings = wins.merge(losses, on=["year_id", "team_id"], how="outer") \
                .merge(draws, on=["year_id", "team_id"], how="outer")

standings = standings.fillna(0)
standings["total_games"] = standings["wins"] + standings["losses"] + standings["draws"]

# 勝率計算（引き分け0.5扱いにする場合は別式にする）
standings["win_pct"] = standings["wins"] / (standings["wins"] + standings["losses"])

# 勝率ランキング
standings = standings.sort_values(["year_id", "win_pct"], ascending=[True, False])

print(standings)


    year_id  team_id  wins  losses  draws  total_games   win_pct
1     30018        2    79      54     19          152  0.593985
4     30018        5    77      58     10          145  0.570370
9     30018       11    74      59     19          152  0.556391
8     30018        9    68      59     21          148  0.535433
11    30018      376    66      63     16          145  0.511628
0     30018        1    63      64     21          148  0.496063
10    30018       12    60      62     21          143  0.491803
5     30018        6    63      68     12          143  0.480916
7     30018        8    55      68     20          143  0.447154
6     30018        7    55      70     18          143  0.440000
3     30018        4    55      71     17          143  0.436508
2     30018        3    54      73     16          143  0.425197
13    30019        2    85      63      5          153  0.574324
21    30019       11    83      68      3          154  0.549669
22    30019       12    7